## Correlation Matrix for Binance Universe.

Calculates correlations for a crypto universe. Add symbols as needed.


### Define Universe


In [5]:
CCXT_FUTURES_SYMBOLS = ['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'TRXUSDT', 'ETCUSDT', 'LINKUSDT', 'XLMUSDT',
                        'ADAUSDT', 'ONTUSDT', 'IOTAUSDT', 'BATUSDT']

### Fetch Prices


In [ ]:

import ccxt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

# Initialize the exchange
exchange = ccxt.binance({
    'rateLimit': 1200,
    'enableRateLimit': True
})

# Function to fetch historical data for a symbol


def fetch_historical_data(symbol):
    # Fetch daily OHLCV data since inception
    # Assuming the exchange started in 2010
    since = exchange.parse8601('2010-01-01T00:00:00Z')
    all_ohlcv = []
    while since:
        ohlcv = exchange.fetch_ohlcv(symbol, '1d', since)
        if ohlcv:
            all_ohlcv.extend(ohlcv)
            since = ohlcv[-1][0] + 1
        else:
            break
    # Convert to DataFrame
    df = pd.DataFrame(all_ohlcv, columns=[
                      'timestamp', 'open', 'high', 'low', 'close', 'volume'])
    # Convert timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    # Set the timestamp as the index
    df.set_index('timestamp', inplace=True)
    return df['close']


# Fetch historical close prices for all symbols
close_prices = {}
for symbol in CCXT_FUTURES_SYMBOLS:
    try:
        close_prices[symbol] = fetch_historical_data(symbol)
        print(f"Fetched data for {symbol}")
    except Exception as e:
        print(f"Error fetching data for {symbol}: {str(e)}")

# Combine close prices into a single DataFrame
combined_df = pd.DataFrame(close_prices)

Fetched data for BTCUSDT
Fetched data for ETHUSDT
Fetched data for BCHUSDT
Fetched data for XRPUSDT
Fetched data for EOSUSDT
Fetched data for LTCUSDT
Fetched data for TRXUSDT


### Calculate Correlations


In [ ]:
# Calculate the correlation matrix
correlation_matrix = combined_df.corr()

# Set the theme to darkgrid for a dark background
sns.set_theme(style="darkgrid")

# Customize the background to black
plt.rcParams['axes.facecolor'] = 'black'
plt.rcParams['figure.facecolor'] = 'black'
plt.rcParams['savefig.facecolor'] = 'black'

# Create a custom diverging palette with red for low and green for high correlation
cmap = sns.diverging_palette(10, 150, s=99, l=50, as_cmap=True)

# Visualize the correlation matrix using a heatmap
plt.figure(figsize=(10, 8))
plt.title('Correlation Matrix of Close Prices',
          color='white')  # Set title color to white
heatmap = sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap=cmap,
                      # Set annotation text color to white
                      annot_kws={"color": "white"},
                      cbar_kws={'label': 'Correlation',
                                'ticks': [-1, 0, 1]},  # Customize colorbar
                      linewidths=0.5, linecolor='gray', vmin=-1, vmax=1)  # Ensure the color range is correct

# Customize the colorbar
colorbar = heatmap.collections[0].colorbar
colorbar.ax.yaxis.label.set_color('white')  # Set colorbar label color to white
colorbar.ax.tick_params(colors='white')  # Set colorbar ticks color to white

# Set tick labels color to white
heatmap.set_xticklabels(heatmap.get_xticklabels(
), rotation=45, horizontalalignment='right', color='white')
heatmap.set_yticklabels(heatmap.get_yticklabels(), rotation=0, color='white')

plt.show()